# 아파트 실거래가 예측 - https://bit.ly/3v5edhJ

### 부동산 빅데이터와 AI를 이용하여 실거래가를 예측 분석 | 회귀 | RMSE

In [ ]:
import pandas as pd

### 데이터 불러오기

In [ ]:
park = 
park.head()

In [ ]:
park.info()

### 공원 타입 - Label Encoding

In [ ]:
# 고유값 확인


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Label Encoding 호출 후 fit
park_le = 
park_le.

# df 변환
park["park_type"] = park_le.
park["park_type"].head()

### 운동 기구

In [ ]:
# 고유값 확인


In [ ]:
# 운동기구가 하나라도 있으면 1, 하나도 없으면 0 으로 변환
park["is_park_exercise_facility"] = 
park["is_park_exercise_facility"].head()

### 놀이 기구

In [ ]:
# 고유값 확인


In [ ]:
# 놀이 기구가 하나라도 있으면 1, 하나도 없으면 0 으로 변환
park["is_park_entertainment_facility"] = 
park["is_park_entertainment_facility"].head()

### 편의 시설

In [ ]:
# 고유값 확인


In [ ]:
# 편의 시설이 하나라도 있으면 1, 하나도 없으면 0 으로 변환
park["is_park_benefit_facility"] = 
park["is_park_benefit_facility"].head()

### 문화 시설

In [ ]:
# 고유값 확인


In [ ]:
# 문화 시설이 하나라도 있으면 1, 하나도 없으면 0 으로 변환
park["is_park_cultural_facitiy"] = 
park["is_park_cultural_facitiy"].head()

### 기타 시설

In [ ]:
# 고유값 확인


In [ ]:
# 기타 시설이 하나라도 있으면 1, 하나도 없으면 0 으로 변환
park["is_park_facility_other"] = 
park["is_park_facility_other"].head()

In [ ]:
park.head()

### 운동 기구, 놀이 기구, 편의 시설, 문화 시설, 기타 시설 drop

In [ ]:

park.head(10)

### train, test와 합치기 위해 city+dong

In [ ]:
park["city_dong"] = 

### 불필요한 column 삭제

In [ ]:

park.head()

### 각 동마다 공원 정보 평균치

In [ ]:
# city_dong으로 groupby, 연산은 평균
# nan값은 0으로 대치
park = 
park.head()

### train, test 데이터 불러오기

In [ ]:
train_df 
test_df
print(train_df.shape)
train_df.head()

In [ ]:
# 데이터 줄이기

train_df = train_df.loc[train_df["transaction_id"]%50==0, :]
train_df = train_df.reset_index(drop=True)
print(train_df.shape)
train_df.head()

In [ ]:
print(test_df.shape)
test_df.head()

In [ ]:
train_df.info()

In [ ]:
test_df.info()

### park와 합치기 위해 city+dong

In [ ]:
train_df["city_dong"] 
test_df["city_dong"] 

### 데이터 프레임 join

In [ ]:
# park 데이터프레임을 "city_dong" 컬럼을 기준으로 각각 train과 test에 join
# nan 값은 0으로 대치
train_df = 
test_df = 

In [ ]:
train_df.head()

In [ ]:
test_df.head()

## 아래는 02와 동일

## city

In [ ]:
train_df["city"].unique()

In [ ]:
test_df["city"].unique()

### Labeling

In [ ]:
train_df.loc[train_df["city"] == "서울특별시", "city"] = 0
train_df.loc[train_df["city"] == "부산광역시", "city"] = 1
train_df["city"] = train_df["city"].astype('int')

test_df.loc[test_df["city"] == "서울특별시", "city"] = 0
test_df.loc[test_df["city"] == "부산광역시", "city"] = 1
test_df["city"] = test_df["city"].astype('int')

In [ ]:
train_df.head()

## drop addr_kr, city_dong

In [ ]:
train_df.drop(['addr_kr', 'city_dong'], axis=1, inplace=True)
test_df.drop(['addr_kr', 'city_dong'], axis=1, inplace=True)

## 한강이 보이는 동네

In [ ]:
hanriver_dong = ['성수동1가','삼성동','이촌동','공덕동','서교동','한강로3가','목동','고덕동', '암사동', '천호동', '풍납동',
                '잠실동', '청담동', '압구정동', '신사동', '잠원동', '반포동', '흑석동', '노량진동', '여의동', '당산동',
                '양평동', '염창동', '가양동', '방화동']

# isin을 사용해서 한강이 보이는 동네는 1, 아니면 0
# True, False를 integer로 변환

train_df['hanriver']=train_df['dong'].isin(hanriver_dong).astype(int)
test_df['hanriver']=test_df['dong'].isin(hanriver_dong).astype(int)

## 평균 실거래가로 동네 sorting

In [ ]:
# "dong"으로 groupby 후, 평균값으로 집계
# 실거래가를 기준으로 오름차순 정렬 후 reset_index()
dong = train_df[['transaction_real_price','dong']].groupby('dong').mean().sort_values('transaction_real_price').reset_index()

# dong의 인덱스를 idx 컬럼에 추가
dong["idx"] = dong.index
dong

In [ ]:
# dong_rank에 동 이름을 key, 순서를 value
dong_rank = {}
for i in range(len(dong)):
    dong_rank[dong["dong"].iloc[i]] = dong["idx"].iloc[i]

In [ ]:
# df의 dong_rank에 위 dong_rank 딕셔너리 매핑
train_df["dong_rank"] = train_df["dong"].map(dong_rank)
test_df["dong_rank"] = test_df["dong"].map(dong_rank)

# dong 컬럼 drop
train_df.drop("dong", axis=1, inplace=True)
test_df.drop("dong", axis=1, inplace=True)

train_df.head()

## 평균 실거래가로 아파트 sorting

In [ ]:
apt = train_df[['transaction_real_price','apt']].groupby('apt').mean().sort_values('transaction_real_price').reset_index()
apt["idx"] = apt.index


apt_rank = {}
for i in range(len(apt)):
    apt_rank[apt["apt"].iloc[i]] = apt["idx"].iloc[i]

train_df["apt_rank"] = train_df["apt"].map(apt_rank)
test_df["apt_rank"] = test_df["apt"].map(apt_rank)

train_df.drop("apt", axis=1, inplace=True)
test_df.drop("apt", axis=1, inplace=True)

train_df.head()

## 평균 실거래가로 지번 sorting

In [ ]:
jibun = train_df[['transaction_real_price','jibun']].groupby('jibun').mean().sort_values('transaction_real_price').reset_index()
jibun["idx"] = jibun.index


jibun_rank = {}
for i in range(len(jibun)):
    jibun_rank[jibun["jibun"].iloc[i]] = jibun["idx"].iloc[i]

train_df["jibun_rank"] = train_df["jibun"].map(jibun_rank)
test_df["jibun_rank"] = test_df["jibun"].map(jibun_rank)

train_df.drop("jibun", axis=1, inplace=True)
test_df.drop("jibun", axis=1, inplace=True)

train_df.head()

In [ ]:
train_df.head()

## 거래 날짜 Labeling

In [ ]:
train_df['transaction_date'].unique(), test_df['transaction_date'].unique()

In [ ]:
date_range = {'1~10': 0, '11~20': 1, '21~28': 2, '21~29': 2, '21~30': 2, '21~31': 2}

train_df.replace({"transaction_date": date_range}, inplace=True)
test_df.replace({"transaction_date": date_range}, inplace=True)

train_df.head()

In [ ]:
train_df.info(), test_df.info()

## test_df null 값 처리

In [ ]:
test_df["dong_rank"].fillna(test_df["dong_rank"].mean(), inplace=True)
test_df["apt_rank"].fillna(test_df["apt_rank"].mean(), inplace=True)
test_df["jibun_rank"].fillna(test_df["jibun_rank"].mean(), inplace=True)

## 데이터셋 분리

In [ ]:
from sklearn.model_selection import train_test_split

x = 
y = 



print(x_train.shape, x_valid.shape)
print(y_train.shape, y_valid.shape)

## 모델 학습 및 예측

### RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf =
rf


y_hat = rf

In [ ]:
rf_rmse = mean_squared_error(y_valid, y_hat)**0.5
rf_rmse

In [ ]:
y_test = 

In [ ]:
rf_submission = pd.concat([test_df['transaction_id'], y_test], axis=1)


rf_submission.columns = ['transaction_id', 'transaction_real_price']
rf_submission.to_csv('rf_park_50_submission.csv', header = True, index = False)

In [ ]:
test = pd.read_csv('rf_park_50_submission.csv')
test

In [ ]:
len(test["transaction_real_price"].unique())

## RandomSearch

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [ ]:
param_distribs = {'n_estimators': randint(low=1, high=200),'max_features': randint(low=1, high=8)}

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)

rnd_search.fit(x_train, y_train)

# 최적의 모델 설정 확인  
rnd_search.best_estimator_

In [ ]:
y_hat = rnd_search.predict(x_valid)

rnd_rf_rmse = mean_squared_error(y_valid, y_hat)**0.5
rnd_rf_rmse

In [ ]:
y_test = pd.Series(rnd_search.predict(test_df))

rnd_rf_submission = pd.concat([test_df['transaction_id'], y_test], axis=1)


rnd_rf_submission.columns = ['transaction_id', 'transaction_real_price']
rnd_rf_submission.to_csv('rnd_rf_park_50_submission.csv', header = True, index = False)

In [ ]:
test = pd.read_csv('rnd_rf_park_50_submission.csv')
test